RNN ------(案例一)文本分类

In [4]:
# -----------------导入库文件-------------
import pandas as pd
import torch
import torch.nn as nn

In [2]:
# --------读文件--------
data = pd.read_csv('sst2/data.csv')

In [3]:
# --------重写数据集------------
class Dataset(torch.utils.data.Dataset):
    def __len__(self):
        return len(data)
    def __getitem__(self,i):
        x, y = data.iloc[i]
        x = [int(i) for i in x.split(',')]
        x = torch.LongTensor(x)
        y = int(y)
        return x, y
# -----------------------------------------
dataset = Dataset()

In [6]:
# -------------数据加载-----------------------
loader = torch.utils.data.DataLoader(dataset = dataset,
                                      batch_size =8,
                                      shuffle = True,
                                      drop_last = True)

In [7]:
# --------定义模型（GRU）-----------
class RNN(nn.Module):
    def __init__(self):
        super().__init__()
        # 词编码
        self.embed = nn.Embedding(num_embeddings=30522, embedding_dim=100)
        # GRU单元
        self.cell = nn.GRUCell(100, 512)
        # 线性输出
        self.fc = nn.Linear(in_features=512, out_features=2)

    def forward(self, x):
        x = self.embed(x)
        h = None  # 初始化记忆
        for i in range(x.shape[1]):
            h = self.cell(x[:, i], h)
        return self.fc(h)
# ------------------------------------------------------------------
rnn = RNN()

In [8]:
# ---------------训练------------------------------------
def train():
    optimizer = torch.optim.Adam(rnn.parameters())
    loss_function = nn.CrossEntropyLoss()
    rnn.train()

    # 定义设备
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    rnn.to(device)

    print('device=', device)
    print('-------------------------------')
    print("|轮数|\t|ID|\t\t|损失|\t\t\t|正确率|")
    for epoch in range(2):
        for i, (x, y) in enumerate(loader):
            # 数据放到GPU里
            x, y = x.to(device), y.to(device)

            out = rnn(x)
            loss = loss_function(out, y)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            if i % 100 == 0:
                acc = out.argmax(dim=1).eq(y).sum().item() / len(y)
                print(epoch, '\t''\t', i, '\t''\t', loss.item(), '\t', acc)
# ------------------------------------------------------------------
train()

device= cuda
-------------------------------
|轮数|	|ID|		|损失|			|正确率|
0 		 0 		 0.7316243648529053 	 0.125
0 		 100 		 0.7051997780799866 	 0.625
0 		 200 		 0.7404813766479492 	 0.5
0 		 300 		 0.6087847948074341 	 0.875
0 		 400 		 0.6649903059005737 	 0.625
0 		 500 		 0.6888590455055237 	 0.75
0 		 600 		 0.9119990468025208 	 0.375
0 		 700 		 0.6251712441444397 	 0.75
0 		 800 		 0.49161988496780396 	 0.75
0 		 900 		 0.5671373605728149 	 0.625
0 		 1000 		 0.559282124042511 	 0.625
0 		 1100 		 0.39387616515159607 	 0.875
0 		 1200 		 0.48385515809059143 	 0.875
0 		 1300 		 0.4911379814147949 	 0.75
0 		 1400 		 0.32054373621940613 	 0.875
0 		 1500 		 0.41746819019317627 	 0.875
0 		 1600 		 0.33436980843544006 	 1.0
0 		 1700 		 0.3389493525028229 	 0.875
0 		 1800 		 0.6434721946716309 	 0.625
0 		 1900 		 0.6135827302932739 	 0.625
0 		 2000 		 0.5219053626060486 	 0.75
0 		 2100 		 0.5939995050430298 	 0.75
0 		 2200 		 0.5740395188331604 	 0.75
0 		 2300 		 0.19398593902587